# 3x3 assembly inputfile that: 

### Import module/Set up XS path/Create openMC model/Clean directory

In [12]:
import sys
import os 
import openmc
import numpy as np
import matplotlib.pyplot as plt
import openmc.mgxs as mgxs
import pandas as pd
from IPython.display import Image
import glob
import re

### Pablo environnement

In [13]:
model=openmc.Model()

clean_file_list = ["model.xml", "materials.xml", "geometry.xml","plots.xml","geometry.xml","settings.xml","tallies.out","summary.h5","statepoint.50.h5"]
for file in clean_file_list :
    path_file = os.path.join(os.getcwd(), file)
    if os.path.exists(path_file):
        os.remove(path_file)

### Define Materials

In [14]:
uo2 = openmc.Material(name='uo2')
uo2.add_nuclide('U234', 6.11864E-06, 'ao')
uo2.add_nuclide('U235', 7.18132E-04, 'ao')
uo2.add_nuclide('U236', 3.29861E-06, 'ao')
uo2.add_nuclide('U238', 2.21546E-02, 'ao')
uo2.add_nuclide('O16', 4.57642E-02, 'ao')
uo2.set_density('g/cm3', 10.257 )
uo2.temperature=565

zirconium = openmc.Material(name="zirconium")
zirconium.add_nuclide('Zr90', 2.18865E-02, 'ao')
zirconium.add_nuclide('Zr91', 4.77292E-03, 'ao')
zirconium.add_nuclide('Zr92', 7.29551E-03, 'ao')
zirconium.add_nuclide('Zr94', 7.39335E-03, 'ao')
zirconium.add_nuclide('Zr96', 1.19110E-03, 'ao')
zirconium.add_nuclide('Sn112',4.68066E-06, 'ao')
zirconium.add_nuclide('Sn114', 3.18478E-06, 'ao')
zirconium.add_nuclide('Sn115', 1.64064E-06, 'ao')
zirconium.add_nuclide('Sn116', 7.01616E-05, 'ao')
zirconium.add_nuclide('Sn117', 3.70592E-05, 'ao')
zirconium.add_nuclide('Sn118', 1.16872E-04, 'ao')
zirconium.add_nuclide('Sn119', 4.14504E-05, 'ao')
zirconium.add_nuclide('Sn120', 1.57212E-04, 'ao')
zirconium.add_nuclide('Sn122', 2.23417E-05, 'ao')
zirconium.add_nuclide('Sn124', 2.79392E-05, 'ao')
zirconium.add_nuclide('Fe54', 8.68307E-06, 'ao')
zirconium.add_nuclide('Fe56', 1.36306E-04, 'ao')
zirconium.add_nuclide('Fe57', 3.14789E-06, 'ao')
zirconium.add_nuclide('Fe58', 4.18926E-07, 'ao')
zirconium.add_nuclide('Cr50', 3.30121E-06, 'ao')
zirconium.add_nuclide('Cr52', 6.36606E-05, 'ao')
zirconium.add_nuclide('Cr53', 7.21860E-06, 'ao')
zirconium.add_nuclide('Cr54', 1.79686E-06, 'ao')
zirconium.add_nuclide('Hf174', 3.54138E-09, 'ao')
zirconium.add_nuclide('Hf176', 1.16423E-07, 'ao')
zirconium.add_nuclide('Hf177', 4.11686E-07, 'ao')
zirconium.add_nuclide('Hf178', 6.03806E-07, 'ao')
zirconium.add_nuclide('Hf179', 3.01460E-07, 'ao')
zirconium.add_nuclide('Hf180', 7.76449E-07, 'ao')
zirconium.set_density('g/cm3',  6.56)

water = openmc.Material(name="water")
water.add_nuclide('H1',4.41459E-02, 'ao')
water.add_nuclide('O16', 2.20729E-02, 'ao')
water.add_nuclide('B10', 9.52537E-06, 'ao')
water.add_nuclide('B11', 3.83408E-05, 'ao')
water.temperature=565
water.set_density('g/cm3', 0.743)
water.add_s_alpha_beta('c_H_in_H2O')

helium = openmc.Material(name="helium")
helium.add_nuclide('He4',1, 'ao')
helium.temperature=565
helium.set_density('g/cm3', 0.17860E-03)



model.materials = openmc.Materials([uo2, zirconium, water,helium]) 
#Perso path
#model.materials.cross_sections = '/home/pablo/internship/xs_for_input/cross_sections.xml'
#Mac path
model.materials.cross_sections = '/Users/pablogarcia44/repo/endfb-vii.1-hdf5/cross_sections.xml'

### Create pincell

In [15]:
def pincell(index_x,index_y):
    fuel_outer_radius = openmc.ZCylinder(r=0.4096)
    clad_inner_radius = openmc.ZCylinder(r=0.418)
    clad_outer_radius = openmc.ZCylinder(r=0.475)
    pitch = 1.26
    left = openmc.XPlane(-pitch/2, boundary_type='transmission')
    right = openmc.XPlane(pitch/2, boundary_type='transmission')
    bottom = openmc.YPlane(-pitch/2, boundary_type='transmission')
    top = openmc.YPlane(pitch/2, boundary_type='transmission')
    fuel_region = -fuel_outer_radius
    gap_region = +fuel_outer_radius & -clad_inner_radius
    clad_region = +clad_inner_radius & -clad_outer_radius
    water_region = +left & -right & +bottom & -top & +clad_outer_radius
    fuel = openmc.Cell(name='fuel'+'_'+str(index_x)+'_'+str(index_y))
    fuel.fill = uo2
    fuel.region = fuel_region
    gap = openmc.Cell(name='gap'+'_'+str(index_x)+'_'+str(index_y))
    gap.region = gap_region
    gap.fill = helium
    clad = openmc.Cell(name='clad'+'_'+str(index_x)+'_'+str(index_y))
    clad.fill = zirconium
    clad.region = clad_region
    moderator = openmc.Cell(name='moderator'+'_'+str(index_x)+'_'+str(index_y))
    moderator.fill = water
    moderator.region = water_region 
    u = openmc.Universe(name='u'+'_'+str(index_x)+'_'+str(index_y),cells=(fuel, gap, clad, moderator))
    return(u,fuel,gap,clad,moderator)

### Create guide tube

In [16]:
def guide(index_x,index_y):
    clad_inner_radius = openmc.ZCylinder(r=0.561)
    clad_outer_radius = openmc.ZCylinder(r=0.602)
    pitch = 1.26
    left = openmc.XPlane(-pitch/2, boundary_type='transmission')
    right = openmc.XPlane(pitch/2, boundary_type='transmission')
    bottom = openmc.YPlane(-pitch/2, boundary_type='transmission')
    top = openmc.YPlane(pitch/2, boundary_type='transmission')       
    clad_region = +clad_inner_radius & -clad_outer_radius
    water_region = +left & -right & +bottom & -top & +clad_outer_radius
    water_guide_region= -clad_inner_radius
    water_guide = openmc.Cell(name='water'+'_'+str(index_x)+'_'+str(index_y))
    water_guide.fill = water
    water_guide.region = water_guide_region
    clad = openmc.Cell(name='clad'+'_'+str(index_x)+'_'+str(index_y))
    clad.fill = zirconium
    clad.region = clad_region
    moderator = openmc.Cell(name='moderator'+'_'+str(index_x)+'_'+str(index_y))
    moderator.fill = water
    moderator.region = water_region 
    u = openmc.Universe(name='u'+'_'+str(index_x)+'_'+str(index_y),cells=(water_guide, clad, moderator))
    return(u,water_guide,water_guide,clad,moderator)

### Create instrumentation tube

In [17]:
def instru(index_x,index_y):
    clad_inner_radius = openmc.ZCylinder(r=0.559)
    clad_outer_radius = openmc.ZCylinder(r=0.605)
    pitch = 1.26
    left = openmc.XPlane(-pitch/2, boundary_type='transmission')
    right = openmc.XPlane(pitch/2, boundary_type='transmission')
    bottom = openmc.YPlane(-pitch/2, boundary_type='transmission')
    top = openmc.YPlane(pitch/2, boundary_type='transmission')       
    clad_region = +clad_inner_radius & -clad_outer_radius
    water_region = +left & -right & +bottom & -top & +clad_outer_radius
    water_guide_region= -clad_inner_radius
    water_guide = openmc.Cell(name='water'+'_'+str(index_x)+'_'+str(index_y))
    water_guide.fill = water
    water_guide.region = water_guide_region
    clad = openmc.Cell(name='clad'+'_'+str(index_x)+'_'+str(index_y))
    clad.fill = zirconium
    clad.region = clad_region
    moderator = openmc.Cell(name='moderator'+'_'+str(index_x)+'_'+str(index_y))
    moderator.fill = water
    moderator.region = water_region 
    u = openmc.Universe(name='u'+'_'+str(index_x)+'_'+str(index_y),cells=(water_guide, clad, moderator))
    return(u,water_guide,water_guide,clad,moderator)

### Mappping

In [18]:
GT=[(5,2),(8,2),(3,3),(2,5),(5,5),(8,5),(2,8),(5,8)]
IT=[(8,8)]

GT_left=[]
GT_full=[]
for X in GT:
    GT_left.append((X[0],X[1]))
    GT_left.append((X[0],16-X[1]))
for X in GT_left:
    GT_full.append((X[0],X[1]))
    GT_full.append((16-X[0],X[1]))    
GT_full = list(set(GT_full))

### Define assembly

In [19]:
assembly = openmc.RectLattice()
pitch=1.26

dr=4e-2 # cm of water that is outside assembly
size=17 #size of the assembly 
pitch_assembly=size*pitch+2*dr 
assembly.pitch = (pitch,pitch)

assembly.lower_left = (-size/2*pitch, -size/2*pitch)


A= np.empty((size, size), dtype=openmc.universe.Universe)
   

for ix in range(size):
    for iy in range(size):
        if (ix,iy) in GT_full : 
            A[ix][iy]=guide(ix,iy)[0]
            A[iy][ix]=A[ix][iy]
        elif (ix,iy) in IT :
            A[ix][iy]=instru(ix,iy)[0]
            A[iy][ix]=A[ix][iy]
        else:
            A[ix][iy]=pincell(ix,iy)[0]
            A[iy][ix]=A[ix][iy]
print(A)   
assembly.universes = A

moderator_outside = openmc.Cell(name='water_outside')
moderator_outside.fill = water
all_water = openmc.Universe()
all_water.add_cell(moderator_outside)
assembly.outer=all_water

rod_height=385.1
# 1/4 assembly
min_x = openmc.XPlane(x0=-(size/2*pitch+dr), boundary_type='reflective')
max_x = openmc.XPlane(x0=0, boundary_type='reflective')
min_y = openmc.YPlane(y0=0, boundary_type='reflective')
max_y = openmc.YPlane(y0=+(size/2*pitch+dr), boundary_type='reflective')
min_z = openmc.ZPlane(z0=-rod_height/2, boundary_type='reflective')
max_z = openmc.ZPlane(z0=+rod_height/2, boundary_type='reflective')



root_cell = openmc.Cell(name='root cell', fill=assembly)
root_cell.region = +min_x & -max_x & +min_y & -max_y & +min_z & -max_z

model.geometry.root_universe = openmc.Universe(name='root universe')
model.geometry.root_universe.add_cell(root_cell)

[[Universe
  	ID             =	294
  	Name           =	u_0_0
  	Geom           =	CSG
  	Cells          =	[1134, 1135, 1136, 1137]
  Universe
  	ID             =	311
  	Name           =	u_1_0
  	Geom           =	CSG
  	Cells          =	[1202, 1203, 1204, 1205]
  Universe
  	ID             =	328
  	Name           =	u_2_0
  	Geom           =	CSG
  	Cells          =	[1270, 1271, 1272, 1273]
  Universe
  	ID             =	345
  	Name           =	u_3_0
  	Geom           =	CSG
  	Cells          =	[1335, 1336, 1337, 1338]
  Universe
  	ID             =	362
  	Name           =	u_4_0
  	Geom           =	CSG
  	Cells          =	[1401, 1402, 1403, 1404]
  Universe
  	ID             =	379
  	Name           =	u_5_0
  	Geom           =	CSG
  	Cells          =	[1469, 1470, 1471, 1472]
  Universe
  	ID             =	396
  	Name           =	u_6_0
  	Geom           =	CSG
  	Cells          =	[1532, 1533, 1534, 1535]
  Universe
  	ID             =	413
  	Name           =	u_7_0
  	Geom           =	CSG
  	Ce

### Plot

model.materials.export_to_xml()
model.geometry.export_to_xml()
plot = openmc.Plot.from_geometry(model.geometry)
plot.pixels = (500, 500)
plot.width = (pitch_assembly+dr, pitch_assembly+dr)
plot.origin = (0., 0., 0)
plot.color_by = 'cell'
plot.to_ipython_image()

### Choose settings

In [20]:
bounds = [-pitch_assembly/2, 0, -rod_height/2, 0, +pitch_assembly/2, rod_height/2]
uniform_dist = openmc.stats.Box(bounds[:3], bounds[3:], only_fissionable=True)
source = openmc.IndependentSource(space=uniform_dist)
source.time = openmc.stats.Uniform(0, 1e-6)
settings = openmc.Settings()
settings.source = source
settings = openmc.Settings()
settings.source = source
settings.batches = 50
settings.inactive = 20
settings.particles = 50000
settings.keff_trigger = {'type':'std_dev','threshold':0.00050}
settings.trigger_active = True
settings.trigger_max_batches = 50000
settings.output = {'tallies':True}
settings.temperature['method'] = 'interpolation'
model.settings = settings

### Define and choose energy group

In [21]:
choose_group=openmc.mgxs.GROUP_STRUCTURES['XMAS-172']
name_group='XMAS172'

### Create MGXS tallies for mgxs directory

In [22]:
pattern = r"[a-zA-Z]+_(\d{1,2})_(\d{1,2})"
def check_numbers(string):
    match = re.match(pattern, string)
    if match:
        n1, n2 = match.groups()
        n1, n2 = int(n1), int(n2)
        return 0 <= n1 <= 8 and 0 <= n2 <= 8
    return False
cell_in_quarter=[]    
for cell in model.geometry.get_all_material_cells().values():
    if check_numbers(cell.name)==True:
        cell_in_quarter.append(cell)       
print(cell_in_quarter)    

[Cell
	ID             =	1134
	Name           =	fuel_0_0
	Fill           =	Material 5
	Region         =	-2005
	Rotation       =	None
	Temperature    =	None
	Translation    =	None
	Volume         =	None
, Cell
	ID             =	1135
	Name           =	gap_0_0
	Fill           =	Material 8
	Region         =	(2005 -2006)
	Rotation       =	None
	Temperature    =	None
	Translation    =	None
	Volume         =	None
, Cell
	ID             =	1136
	Name           =	clad_0_0
	Fill           =	Material 6
	Region         =	(2006 -2007)
	Rotation       =	None
	Temperature    =	None
	Translation    =	None
	Volume         =	None
, Cell
	ID             =	1137
	Name           =	moderator_0_0
	Fill           =	Material 7
	Region         =	(2008 -2009 2010 -2011 2007)
	Rotation       =	None
	Temperature    =	None
	Translation    =	None
	Volume         =	None
, Cell
	ID             =	1202
	Name           =	fuel_1_0
	Fill           =	Material 5
	Region         =	-2124
	Rotation       =	None
	Temperature    =	N

In [23]:
mgxs_lib = openmc.mgxs.Library(model.geometry)
groups = openmc.mgxs.EnergyGroups(choose_group)
mgxs_lib.energy_groups = groups
mgxs_lib.scatter_format = "legendre"
mgxs_lib.legendre_order = 3
mgxs_lib.mgxs_types = ['total', 'absorption', 'fission', 'nu-fission', 'chi', 'scatter matrix', 'nu-scatter matrix', 'inverse-velocity']
#mgxs_lib.tally_trigger =openmc.Trigger('std_dev', 1e-2)
mgxs_lib.by_nuclide = False
mgxs_lib.domain_type = 'cell'
mgxs_lib.domains = cell_in_quarter
mgxs_lib.build_library()
tallies = openmc.Tallies()
mgxs_lib.add_to_tallies_file(tallies, merge=True)
model.tallies = tallies

#model.export_to_xml()
# model.export_to_model_xml('model_2A_36_families.xml')
model.export_to_model_xml('model_36_test.xml')

/opt/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mgxs/library.py:401: RuntimeWarning: The P0 correction will be ignored since the scattering order 3 is greater than zero
  warn(msg, RuntimeWarning)
/opt/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Filter instance already exists with id=6422.
  warn(msg, IDWarning)
/opt/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Filter instance already exists with id=2.
  warn(msg, IDWarning)
/opt/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Filter instance already exists with id=10.
  warn(msg, IDWarning)


### Create power tally

tally_power = openmc.Tally(name='power')
tally_flux = openmc.Tally(name='flux')

all_cell=model.geometry.get_all_material_cells().values()
A=[]
B=[]
for cell in all_cell:
    for x in range(9):
        for y in range(9):
            if cell.name == 'fuel_'+str(x)+'_'+str(y):
                B.append(cell)
    A.append(cell)
    
fuel_cell=openmc.CellFilter(A)
all_cell=openmc.CellFilter(A)

tally_power.scores = ['kappa-fission']
tally_power.filters = [fuel_cell]
#trigger_power = openmc.Trigger('std_dev', 1000)
#tally_power.triggers = [trigger_power]


energy_filter = openmc.EnergyFilter([0.0, 4.0, 1.0e6])
tally_flux.scores = ['flux']
tally_flux.filters = [all_cell,energy_filter]


tallies = openmc.Tallies([tally_power,tally_flux])
model.tallies=tallies


#model.export_to_xml()
model.export_to_model_xml('model_2A_72_families_power.xml')

### Run OpenMC model

sp_file = model.run()